# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"],
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [143, 156, 1134, 991, 27],
                                      [143, 156, 1134, 991, 27]
                                  ],
             "bought_list": [
                             [156, 27, 33, 42],
                             [156, 143],
                             [1, 11]
                            ]})
data

,user_id,recommended_list,bought_list
0,u1,"[143, 156, 1134, 991, 27]","[156, 27, 33, 42]"
1,u2,"[143, 156, 1134, 991, 27]","[156, 143]"
2,u3,"[143, 156, 1134, 991, 27]","[1, 11]"


In [6]:
prices = pd.DataFrame({"item_id": [143, 156, 1134, 991, 27, 33, 42, 1, 11],
                       "price": [400, 60, 40, 40, 90, 30, 20, 10, 100]})
prices

,item_id,price
0,143,400
1,156,60
2,1134,40
3,991,40
4,27,90
5,33,30
6,42,20
7,1,10
8,11,100


In [7]:
prices_resommended = [400, 60, 40, 40, 90, 30, 20, 10, 100] #143, 156, 1134, 991, 27, 33, 42, 1, 11

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [34]:
def recall_at_k(recommended_list, bought_list, k = 4):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list[:k])
    recall = flags.sum() / len(bought_list)
    return recall


def money_recall_at_k(recommended_list, bought_list, prices, k = 4):
    all_revenue = get_revenue = 0

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list[:k])

    for i in range(len(bought_list)):
      if flags[i]:
        get_revenue += int(prices[prices['item_id'] == bought_list[i]].price)

    for i in range(len(bought_list)):
      all_revenue += int(prices[prices['item_id'] == bought_list[i]].price)

    recall = get_revenue / all_revenue

    return recall

In [31]:
data.apply(lambda x: recall_at_k(x[1], x[2]), 1)

0    0.25
1    1.00
2    0.00
dtype: float64

In [35]:
data.apply(lambda x: money_recall_at_k(x[1], x[2], prices), 1)

0    0.3
1    1.0
2    0.0
dtype: float64

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [48]:
def mrr_at_k(recommended_list, bought_list, k = 4):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    rank = 0

    flags = np.isin(recommended_list[:k], bought_list)

    for i in range(len(recommended_list[:k])):
      if flags[i]:
        rank = 1 / (i + 1)
        break

    return rank


In [49]:
data.apply(lambda x: mrr_at_k(x[1], x[2]), 1).mean()

0.5

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [8]:
from math import log

In [50]:
def ndcg_at_k(recommended_list, bought_list, k=4):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    DCG = 0
    IDCG = 0

    flags = np.isin(recommended_list[:k], bought_list)

    for i in range(len(flags)):
      if flags[i] == 1:
        if i == 0:
          DCG += 1
          #print(f"i = {i}, 1")
        else:
          DCG += 1 / log(i + 2)
          #print(f"i = {i}, 1 / log{i + 2}")

    for i in range(len(bought_list)):
      if i == 0:
        IDCG += 1
        #print(f"i = {i}, 1")
      else:
        IDCG += 1 / log(i + 2)

    nDCG = DCG / IDCG

    return nDCG


In [51]:
data.apply(lambda x: ndcg_at_k(x[1], x[2]), 1).mean()

0.4266073513509315